In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import os
import pickle
import threading
import time

# Define the color scheme
NAVY_BLUE = "#0A1931"
NAVY_BLUE_LIGHT = "#1C3259"
CREAM = "#FFF8DC"
CREAM_LIGHT = "#FFFAF0"
ACCENT_COLOR = "#FFC93C"
TEXT_COLOR = "#0A1931"
ERROR_COLOR = "#FF5A5A"
SUCCESS_COLOR = "#4CAF50"

class CyberThreatDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Analysing and Detecting Cyber Threats")
        self.root.geometry("1200x800")
        self.root.configure(bg=NAVY_BLUE)
        
        # Configure styles for ttk widgets
        self.configure_styles()
        
        # Variables
        self.dataset = None
        self.filename = None
        self.X = None
        self.Y = None
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
        self.autoencoder = None
        self.encoder_model = None
        self.decision_tree = None
        self.dnn = None
        self.pca = None
        self.scaler = None
        self.accuracy = []
        self.precision = []
        self.recall = []
        self.fscore = []
        self.vector = None
        self.progress_var = tk.DoubleVar()
        self.status_var = tk.StringVar()
        self.status_var.set("Ready")
        
        # Attack labels
        self.labels = [
            'Normal', 
            '(NMRI)', 
            '(CMRI)', 
            '(MSCI)',
            ' (MPCI)', 
            '(MFCI)', 
            ' (DoS)',
            '(MITM)'
        ]
        
        # Create UI
        self.create_widgets()
    
    def configure_styles(self):
        style = ttk.Style()
        style.theme_use('default')
        
        # Configure the Notebook style (tabs)
        style.configure("TNotebook", background=NAVY_BLUE, borderwidth=0)
        style.configure("TNotebook.Tab", background=NAVY_BLUE_LIGHT, foreground=CREAM, padding=[15, 5], font=('Arial', 11))
        style.map("TNotebook.Tab", background=[("selected", CREAM)], foreground=[("selected", NAVY_BLUE)])
        
        # Configure other ttk elements
        style.configure("TFrame", background=CREAM_LIGHT)
        style.configure("TButton", background=NAVY_BLUE, foreground=CREAM, font=('Arial', 11), borderwidth=1)
        style.map("TButton", background=[("active", NAVY_BLUE_LIGHT)])
        style.configure("TLabel", background=CREAM_LIGHT, foreground=TEXT_COLOR, font=('Arial', 11))
        style.configure("TProgressbar", troughcolor=CREAM_LIGHT, background=ACCENT_COLOR)
        
    def create_widgets(self):
        # Header frame
        header_frame = tk.Frame(self.root, bg=NAVY_BLUE, height=80)
        header_frame.pack(fill=tk.X)
        
        title_label = tk.Label(header_frame, 
                            text="Analysing and Detecting Cyber Threats", 
                            font=("Arial", 20, "bold"), 
                            bg=NAVY_BLUE, 
                            fg=CREAM)
        title_label.pack(pady=20)
        
        # Main content frame with tabs
        tab_control = ttk.Notebook(self.root)
        
        # Tab 1: Data Analysis
        data_tab = ttk.Frame(tab_control)
        tab_control.add(data_tab, text="Data Analysis")
        
        # Tab 2: Model Training
        model_tab = ttk.Frame(tab_control)
        tab_control.add(model_tab, text="Model Training")
        
        # Tab 3: Attack Detection
        detection_tab = ttk.Frame(tab_control)
        tab_control.add(detection_tab, text="Attack Detection")
        
        # Tab 4: Results Comparison
        results_tab = ttk.Frame(tab_control)
        tab_control.add(results_tab, text="Results")
        
        tab_control.pack(expand=1, fill="both")
        
        # ----- Data Analysis Tab -----
        # Left panel for controls
        data_left_panel = tk.Frame(data_tab, width=300, bg=CREAM_LIGHT)
        data_left_panel.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)
        
        tk.Label(data_left_panel, text="Dataset Operations", font=("Arial", 14, "bold"), bg=CREAM_LIGHT, fg=NAVY_BLUE).pack(pady=10)
        
        # Button style
        btn_style = {"bg": NAVY_BLUE, "fg": CREAM, "font": ("Arial", 12), "width": 20, "padx": 10, "pady": 5, "borderwidth": 0, "relief": tk.FLAT}
        
        upload_btn = tk.Button(data_left_panel, text="Upload Dataset", 
                            command=self.upload_dataset, 
                            **btn_style)
        upload_btn.pack(pady=10)
        
        preprocess_btn = tk.Button(data_left_panel, text="Preprocess Dataset", 
                              command=self.preprocess_data, 
                              **btn_style)
        preprocess_btn.pack(pady=10)
        
        view_stats_btn = tk.Button(data_left_panel, text="View Attack Distribution", 
                             command=self.show_attack_distribution, 
                             **btn_style)
        view_stats_btn.pack(pady=10)
        
        # Right panel for data display
        data_right_panel = tk.Frame(data_tab, bg=CREAM_LIGHT)
        data_right_panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Create a frame for the text area and scrollbar
        text_frame = tk.Frame(data_right_panel, bg=CREAM_LIGHT)
        text_frame.pack(fill=tk.BOTH, expand=True)
        
        # Text widget with scrollbar
        self.text_area = tk.Text(text_frame, wrap=tk.WORD, font=("Consolas", 11), bg=CREAM, fg=TEXT_COLOR)
        scrollbar = tk.Scrollbar(text_frame, command=self.text_area.yview)
        self.text_area.configure(yscrollcommand=scrollbar.set)
        
        self.text_area.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Frame for the visualization
        self.data_viz_frame = tk.Frame(data_right_panel, bg=CREAM_LIGHT)
        self.data_viz_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # ----- Model Training Tab -----
        model_left_panel = tk.Frame(model_tab, width=300, bg=CREAM_LIGHT)
        model_left_panel.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)
        
        tk.Label(model_left_panel, text="Model Training", font=("Arial", 14, "bold"), bg=CREAM_LIGHT, fg=NAVY_BLUE).pack(pady=10)
        
        autoencoder_btn = tk.Button(model_left_panel, text="Train AutoEncoder", 
                               command=lambda: self.run_model("autoencoder"), 
                               **btn_style)
        autoencoder_btn.pack(pady=10)
        
        dt_btn = tk.Button(model_left_panel, text="Train Decision Tree", 
                        command=lambda: self.run_model("decision_tree"), 
                        **btn_style)
        
        dt_btn.pack(pady=10)
        
        dnn_btn = tk.Button(model_left_panel, text="Train DNN", 
                        command=lambda: self.run_model("dnn"), 
                        **btn_style)
        dnn_btn.pack(pady=10)
        
        # Progress bar and status
        tk.Label(model_left_panel, text="Training Progress:", font=("Arial", 12), bg=CREAM_LIGHT, fg=NAVY_BLUE).pack(pady=(20, 5))
        self.progress_bar = ttk.Progressbar(model_left_panel, variable=self.progress_var, mode="determinate", length=250)
        self.progress_bar.pack(pady=5)
        
        self.status_label = tk.Label(model_left_panel, textvariable=self.status_var, font=("Arial", 10), bg=CREAM_LIGHT, fg=NAVY_BLUE)
        self.status_label.pack(pady=5)
        
        # Right panel for model training results
        model_right_panel = tk.Frame(model_tab, bg=CREAM_LIGHT)
        model_right_panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Text widget for model training results
        self.model_text_area = tk.Text(model_right_panel, wrap=tk.WORD, font=("Consolas", 11), bg=CREAM, fg=TEXT_COLOR)
        model_scrollbar = tk.Scrollbar(model_right_panel, command=self.model_text_area.yview)
        self.model_text_area.configure(yscrollcommand=model_scrollbar.set)
        
        self.model_text_area.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        model_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # ----- Attack Detection Tab -----
        detection_frame = tk.Frame(detection_tab, bg=CREAM_LIGHT)
        detection_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        tk.Label(detection_frame, text="Cyber Attack Detection & Attribution", font=("Arial", 16, "bold"), bg=CREAM_LIGHT, fg=NAVY_BLUE).pack(pady=10)
        
        # Frame for file selection
        file_frame = tk.Frame(detection_frame, bg=CREAM_LIGHT)
        file_frame.pack(fill=tk.X, pady=10)
        
        tk.Label(file_frame, text="Select Test Data:", font=("Arial", 12), bg=CREAM_LIGHT, fg=NAVY_BLUE).pack(side=tk.LEFT, padx=10)
        
        self.file_path_var = tk.StringVar()
        file_entry = tk.Entry(file_frame, textvariable=self.file_path_var, width=50, font=("Arial", 11), bg=CREAM, fg=TEXT_COLOR)
        file_entry.pack(side=tk.LEFT, padx=10)
        
        browse_btn = tk.Button(file_frame, text="Browse", command=self.browse_test_file, bg=NAVY_BLUE, fg=CREAM, font=("Arial", 11))
        browse_btn.pack(side=tk.LEFT, padx=10)
        
        detect_btn = tk.Button(detection_frame, text="Detect & Attribute Attack", command=self.detect_attack, bg=NAVY_BLUE, fg=CREAM, font=("Arial", 12))
        detect_btn.pack(pady=15)
        
        # Results area
        result_frame = tk.Frame(detection_frame, bg=CREAM_LIGHT)
        result_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        self.detection_text = tk.Text(result_frame, wrap=tk.WORD, font=("Consolas", 11), bg=CREAM, fg=TEXT_COLOR)
        detection_scrollbar = tk.Scrollbar(result_frame, command=self.detection_text.yview)
        self.detection_text.configure(yscrollcommand=detection_scrollbar.set)
        
        self.detection_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        detection_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # ----- Results Comparison Tab -----
        results_frame = tk.Frame(results_tab, bg=CREAM_LIGHT)
        results_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        tk.Label(results_frame, text="Performance Comparison", font=("Arial", 16, "bold"), bg=CREAM_LIGHT, fg=NAVY_BLUE).pack(pady=10)
        
        btn_frame = tk.Frame(results_frame, bg=CREAM_LIGHT)
        btn_frame.pack(fill=tk.X, pady=10)
        
        graph_btn = tk.Button(btn_frame, text="Show Comparison Graph", command=self.show_comparison_graph, bg=NAVY_BLUE, fg=CREAM, font=("Arial", 12))
        graph_btn.pack(side=tk.LEFT, padx=10)
        
        table_btn = tk.Button(btn_frame, text="Show Comparison Table", command=self.show_comparison_table, bg=NAVY_BLUE, fg=CREAM, font=("Arial", 12))
        table_btn.pack(side=tk.LEFT, padx=10)
        
        # Visualization frame
        self.results_viz_frame = tk.Frame(results_frame, bg=CREAM_LIGHT)
        self.results_viz_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # Status bar at the bottom
        status_bar = tk.Frame(self.root, bg=NAVY_BLUE, height=25)
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        status_label = tk.Label(status_bar, text="© 2025 Analysing and Detecting Cyber Threats", bg=NAVY_BLUE, fg=CREAM, font=("Arial", 9))
        status_label.pack(side=tk.RIGHT, padx=10)
        
    def upload_dataset(self):
        self.filename = filedialog.askopenfilename(initialdir="Dataset", title="Select Dataset", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
        if self.filename:
            try:
                self.text_area.delete(1.0, tk.END)
                self.dataset = pd.read_csv(self.filename)
                
                self.text_area.insert(tk.END, f"Dataset loaded: {self.filename}\n\n")
                self.text_area.insert(tk.END, "Dataset Preview:\n\n")
                self.text_area.insert(tk.END, str(self.dataset.head()) + "\n\n")
                
                self.text_area.insert(tk.END, "Dataset Information:\n")
                self.text_area.insert(tk.END, f"Total Rows: {self.dataset.shape[0]}\n")
                self.text_area.insert(tk.END, f"Total Columns: {self.dataset.shape[1]}\n\n")
                
                # Show message
                messagebox.showinfo("Success", "Dataset loaded successfully!")
            except Exception as e:
                self.text_area.insert(tk.END, f"Error: {str(e)}")
                messagebox.showerror("Error", f"Failed to load dataset: {str(e)}")
    
    def preprocess_data(self):
        if self.dataset is None:
            messagebox.showwarning("Warning", "Please upload a dataset first!")
            return
        
        try:
            self.text_area.delete(1.0, tk.END)
            
            # Fill NA values
            self.dataset.fillna(0, inplace=True)
            
            # Load or create scaler
            if os.path.exists('model/minmax.txt'):
                with open('model/minmax.txt', 'rb') as file:
                    self.scaler = pickle.load(file)
                self.text_area.insert(tk.END, "Loaded existing MinMaxScaler\n\n")
            else:
                self.scaler = MinMaxScaler()
                # Create directory if it doesn't exist
                os.makedirs('model', exist_ok=True)
                self.text_area.insert(tk.END, "Created new MinMaxScaler\n\n")
            
            # Prepare data
            data_values = self.dataset.values
            self.X = data_values[:, 0:data_values.shape[1]-1]
            self.Y = data_values[:, data_values.shape[1]-1]
            
            # Shuffle data
            indices = np.arange(self.X.shape[0])
            np.random.shuffle(indices)
            self.X = self.X[indices]
            self.Y = self.Y[indices]
            
            # One-hot encode labels
            self.Y = to_categorical(self.Y)
            
            # Normalize features
            self.X = self.scaler.fit_transform(self.X)
            
            # Save scaler
            with open('model/minmax.txt', 'wb') as file:
                pickle.dump(self.scaler, file)
            
            # Split data for training and testing
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.Y, test_size=0.2, random_state=42)
            
            self.text_area.insert(tk.END, "Dataset after normalization (sample):\n")
            self.text_area.insert(tk.END, str(self.X[:5]) + "\n\n")
            
            self.text_area.insert(tk.END, "Training and testing split:\n")
            self.text_area.insert(tk.END, f"Training samples: {self.X_train.shape[0]}\n")
            self.text_area.insert(tk.END, f"Testing samples: {self.X_test.shape[0]}\n")
            
            messagebox.showinfo("Success", "Data preprocessing completed successfully!")
            
        except Exception as e:
            self.text_area.insert(tk.END, f"Error during preprocessing: {str(e)}")
            messagebox.showerror("Error", f"Preprocessing failed: {str(e)}")
    
    def show_attack_distribution(self):
        if self.dataset is None:
            messagebox.showwarning("Warning", "Please upload a dataset first!")
            return
        
        try:
            # Clear previous plots
            for widget in self.data_viz_frame.winfo_children():
                widget.destroy()
            
            # Count attack types
            unique, count = np.unique(self.dataset['result'], return_counts=True)
            
            # Create figure with navy blue background
            fig = plt.Figure(figsize=(30, 30), facecolor=CREAM_LIGHT)
            ax = fig.add_subplot(111)
            
            # Plot
            bars = ax.bar(range(len(count)), count, color=NAVY_BLUE)
            ax.set_xticks(range(len(count)))
            ax.set_xticklabels(self.labels, rotation=0, ha='right', va='center')
            ax.set_title('Distribution of Cyber Attack Types', color=NAVY_BLUE, fontsize=14)
            ax.set_xlabel('Attack Type', color=NAVY_BLUE)
            ax.set_ylabel('Count', color=NAVY_BLUE)
            
            # Add count labels on top of bars
            for i, bar in enumerate(bars):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                        f'{int(height)}',
                        ha='center', va='bottom', color=NAVY_BLUE)
            
            # Style the plot to match the app theme
            ax.set_facecolor(CREAM)
            for spine in ax.spines.values():
                spine.set_color(NAVY_BLUE)
            
            ax.tick_params(colors=NAVY_BLUE)
            
            # Create canvas
            canvas = FigureCanvasTkAgg(fig, self.data_viz_frame)
            canvas.draw()
            canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to generate distribution plot: {str(e)}")
    
    def run_model(self, model_type):
        if self.X_train is None or self.y_train is None:
            messagebox.showwarning("Warning", "Please preprocess the data first!")
            return
        
        # Start a thread for training to avoid UI freeze
        threading.Thread(target=self._train_model, args=(model_type,)).start()
    
    def _train_model(self, model_type):
        try:
            self.model_text_area.delete(1.0, tk.END)
            self.status_var.set(f"Training {model_type}...")
            self.progress_var.set(0)
            
            if model_type == "autoencoder":
                self._train_autoencoder()
            elif model_type == "decision_tree":
                self._train_decision_tree()
            elif model_type == "dnn":
                self._train_dnn()
            
            self.status_var.set(f"{model_type.capitalize()} training completed")
            self.progress_var.set(100)
            messagebox.showinfo("Success", f"{model_type.capitalize()} trained successfully!")
            
        except Exception as e:
            self.model_text_area.insert(tk.END, f"Error during training: {str(e)}")
            self.status_var.set("Training failed")
            messagebox.showerror("Error", f"Failed to train {model_type}: {str(e)}")
    
    def _train_autoencoder(self):
        # Update progress
        self.progress_var.set(10)
        self.root.update_idletasks()
        
        # Build autoencoder model
        input_layer = Input(shape=(self.X.shape[1],))
        
        # Encoder
        encoded = Dense(256, activation='relu')(input_layer)
        self.progress_var.set(20)
        self.root.update_idletasks()
        
        encoded = Dense(128, activation='relu')(encoded)
        self.progress_var.set(30)
        self.root.update_idletasks()
        
        encoded = Dense(64, activation='relu')(encoded)
        self.progress_var.set(40)
        self.root.update_idletasks()
        
        # Decoder
        decoded = Dense(128, activation='relu')(encoded)
        self.progress_var.set(50)
        self.root.update_idletasks()
        
        decoded = Dense(256, activation='relu')(decoded)
        self.progress_var.set(60)
        self.root.update_idletasks()
        
        decoded = Dense(self.X.shape[1], activation='sigmoid')(decoded)
        self.progress_var.set(70)
        self.root.update_idletasks()
        
        # Create and compile model
        self.autoencoder = Model(inputs=input_layer, outputs=decoded)
        self.autoencoder.compile(optimizer='adam', loss='mse')
        
        # Create encoder model
        self.encoder_model = Model(inputs=input_layer, outputs=encoded)
        
        # Save model architecture
        model_json = self.encoder_model.to_json()
        with open("model/encoder_model.json", "w") as json_file:
            json_file.write(model_json)
        
        # Train the model with a small subset for demonstration
        history = self.autoencoder.fit(self.X_train, self.X_train, 
                                    epochs=5, 
                                    batch_size=32, 
                                    shuffle=True,
                                    validation_data=(self.X_test, self.X_test),
                                    verbose=0)
        
        self.progress_var.set(80)
        self.root.update_idletasks()
        
        # Save weights
        self.encoder_model.save_weights("model/encoder_model.weights.h5")
        self.autoencoder.save_weights("model/autoencoder.weights.h5")
        
        self.progress_var.set(90)
        self.root.update_idletasks()
        
        # Evaluate
        loss = self.autoencoder.evaluate(self.X_test, self.X_test, verbose=0)
        
        # Display results
        self.model_text_area.insert(tk.END, "AutoEncoder Training Results:\n\n")
        self.model_text_area.insert(tk.END, f"Model Architecture:\n{self.autoencoder.summary()}\n\n")
        self.model_text_area.insert(tk.END, f"Test Loss: {loss}\n\n")
        
        # Calculate reconstruction error
        predictions = self.autoencoder.predict(self.X_test)
        mse = np.mean(np.power(self.X_test - predictions, 2), axis=1)
        
        self.model_text_area.insert(tk.END, f"Average Reconstruction Error: {np.mean(mse)}\n")
        
        # Store metrics
        self.accuracy.append(100 - np.mean(mse) * 100)  # Just an approximation
        self.precision.append(None)  # Not applicable for autoencoder
        self.recall.append(None)
        self.fscore.append(None)
        
        self.model_text_area.insert(tk.END, "\nAutoEncoder model saved successfully.\n")
        self.progress_var.set(100)
    
    def _train_decision_tree(self):
        if self.encoder_model is None:
            self.model_text_area.insert(tk.END, "Please train AutoEncoder first!")
            return
        
        self.progress_var.set(20)
        self.root.update_idletasks()
        
        # Extract features using autoencoder
        self.vector = self.encoder_model.predict(self.X)
        
        self.progress_var.set(40)
        self.root.update_idletasks()
        
        # Apply PCA
        self.pca = PCA(n_components=7)
        self.vector = self.pca.fit_transform(self.vector)
        
        self.progress_var.set(60)
        self.root.update_idletasks()
        
        # Get the original labels
        Y1 = np.argmax(self.Y, axis=1)
        
        # Split data
        X_train_dt, X_test_dt, y_train_dt, y_test_dt = train_test_split(self.vector, Y1, test_size=0.2)
        
        # Train Decision Tree
        self.decision_tree = DecisionTreeClassifier(max_depth=10, random_state=42)
        self.decision_tree.fit(X_train_dt, y_train_dt)
        
        self.progress_var.set(80)
        self.root.update_idletasks()
        
        # Predict and evaluate
        dt_pred = self.decision_tree.predict(X_test_dt)
        
        # Calculate metrics
        dt_accuracy = accuracy_score(y_test_dt, dt_pred) * 100
        dt_precision = precision_score(y_test_dt, dt_pred, average='macro') * 100
        dt_recall = recall_score(y_test_dt, dt_pred, average='macro') * 100
        dt_fscore = f1_score(y_test_dt, dt_pred, average='macro') * 100
        
        # Store metrics
        self.accuracy.append(dt_accuracy)
        self.precision.append(dt_precision)
        self.recall.append(dt_recall)
        self.fscore.append(dt_fscore)
        
        # Display results
        self.model_text_area.insert(tk.END, "Decision Tree with PCA Training Results:\n\n")
        self.model_text_area.insert(tk.END, f"Features after PCA: {self.vector.shape[1]}\n\n")
        self.model_text_area.insert(tk.END, f"Accuracy: {dt_accuracy:.2f}%\n")
        self.model_text_area.insert(tk.END, f"Precision: {dt_precision:.2f}%\n")
        self.model_text_area.insert(tk.END, f"Recall: {dt_recall:.2f}%\n")
        self.model_text_area.insert(tk.END, f"F1 Score: {dt_fscore:.2f}%\n\n")
        
        # Save model
        with open('model/decision_tree.pkl', 'wb') as f:
            pickle.dump(self.decision_tree, f)
        
        self.model_text_area.insert(tk.END, "Decision Tree model saved successfully.\n")
        self.progress_var.set(100)
    
    def _train_dnn(self):
        if self.decision_tree is None or self.vector is None:
            self.model_text_area.insert(tk.END, "Please train Decision Tree first!")
            return
        
        self.progress_var.set(20)
        self.root.update_idletasks()
        
        # Get predicted attack types from decision tree
        attack_type = []
        for i in range(len(self.vector)):
            temp = []
            temp.append(self.vector[i])
            attack = self.decision_tree.predict(np.asarray(temp))
            attack_type.append(attack[0])
        
        attack_type = np.asarray(attack_type)
        
        self.progress_var.set(40)
        self.root.update_idletasks()
        
        # Split data
        X_train_dnn, X_test_dnn, y_train_dnn, y_test_dnn = train_test_split(self.vector, attack_type, test_size=0.2)
        
        # Train DNN
        self.dnn = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, activation='relu', solver='adam', random_state=42)
        # Train DNN
        self.dnn = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, activation='relu', solver='adam', random_state=42)
        self.dnn.fit(X_train_dnn, y_train_dnn)
        
        self.progress_var.set(80)
        self.root.update_idletasks()
        
        # Predict and evaluate
        dnn_pred = self.dnn.predict(X_test_dnn)
        
        # Calculate metrics
        dnn_accuracy = accuracy_score(y_test_dnn, dnn_pred) * 100
        dnn_precision = precision_score(y_test_dnn, dnn_pred, average='macro') * 100
        dnn_recall = recall_score(y_test_dnn, dnn_pred, average='macro') * 100
        dnn_fscore = f1_score(y_test_dnn, dnn_pred, average='macro') * 100
        
        # Store metrics
        self.accuracy.append(dnn_accuracy)
        self.precision.append(dnn_precision)
        self.recall.append(dnn_recall)
        self.fscore.append(dnn_fscore)
        
        # Display results
        self.model_text_area.insert(tk.END, "DNN Training Results:\n\n")
        self.model_text_area.insert(tk.END, f"Hidden Layer Sizes: (100, 50)\n")
        self.model_text_area.insert(tk.END, f"Max Iterations: 300\n\n")
        self.model_text_area.insert(tk.END, f"Accuracy: {dnn_accuracy:.2f}%\n")
        self.model_text_area.insert(tk.END, f"Precision: {dnn_precision:.2f}%\n")
        self.model_text_area.insert(tk.END, f"Recall: {dnn_recall:.2f}%\n")
        self.model_text_area.insert(tk.END, f"F1 Score: {dnn_fscore:.2f}%\n\n")
        
        # Save model
        with open('model/dnn.pkl', 'wb') as f:
            pickle.dump(self.dnn, f)
        
        self.model_text_area.insert(tk.END, "DNN model saved successfully.\n")
        self.progress_var.set(100)
    
    def browse_test_file(self):
        filename = filedialog.askopenfilename(initialdir="Dataset", title="Select Test Data", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
        if filename:
            self.file_path_var.set(filename)
    
    def detect_attack(self):
        if self.dnn is None or self.encoder_model is None or self.decision_tree is None:
            messagebox.showwarning("Warning", "Please train all models first!")
            return
        
        filepath = self.file_path_var.get()
        if not filepath:
            messagebox.showwarning("Warning", "Please select a test file!")
            return
        
        try:
            # Clear previous results
            self.detection_text.delete(1.0, tk.END)
            
            # Load and preprocess test data
            test_data = pd.read_csv(filepath)
            test_data.fillna(0, inplace=True)
            
            # Store original values for display
            original_values = test_data.values
            
            # Normalize features
            test_features = self.scaler.transform(test_data)
            
            # Extract features using autoencoder
            test_vector = self.encoder_model.predict(test_features)
            
            # Apply PCA
            test_vector = self.pca.transform(test_vector)
            
            # Predict attacks
            predictions = self.dnn.predict(test_vector)
            
            # Display results
            self.detection_text.insert(tk.END, "Attack Detection Results:\n\n")
            
            for i in range(len(predictions)):
                result_text = ""
                if predictions[i] == 0:
                    result_text = f"Test Data {i+1}: {str(original_values[i])}\n"
                    result_text += "Result: NO CYBER ATTACK DETECTED\n\n"
                    self.detection_text.insert(tk.END, result_text)
                    
                    # Add a tag to highlight normal traffic
                    start_pos = self.detection_text.index("end-3l")
                    end_pos = self.detection_text.index("end-2l")
                    self.detection_text.tag_add("normal", start_pos, end_pos)
                    self.detection_text.tag_config("normal", foreground=SUCCESS_COLOR, font=("Arial", 12, "bold"))
                else:
                    result_text = f"Test Data {i+1}: {str(original_values[i])}\n"
                    result_text += f"Result: CYBER ATTACK DETECTED\n"
                    result_text += f"Attack Type: {self.labels[predictions[i]]}\n\n"
                    self.detection_text.insert(tk.END, result_text)
                    
                    # Add a tag to highlight attacks
                    start_pos = self.detection_text.index(f"end-3l")
                    end_pos = self.detection_text.index(f"end-1l")
                    self.detection_text.tag_add("attack", start_pos, end_pos)
                    self.detection_text.tag_config("attack", foreground=ERROR_COLOR, font=("Arial", 12, "bold"))
            
        except Exception as e:
            self.detection_text.insert(tk.END, f"Error during detection: {str(e)}")
            messagebox.showerror("Error", f"Detection failed: {str(e)}")
    
    def show_comparison_graph(self):
        if len(self.accuracy) < 3:
            messagebox.showwarning("Warning", "Please train all models first!")
            return
        
        try:
            # Clear previous charts
            for widget in self.results_viz_frame.winfo_children():
                widget.destroy()
            
            # Create figure with proper colors
            fig = plt.Figure(figsize=(10, 6), facecolor=CREAM_LIGHT)
            ax = fig.add_subplot(111)
            
            # Define models and metrics
            models = ["AutoEncoder", "Decision Tree", "DNN"]
            metrics_data = {
                "Accuracy": self.accuracy,
                "Precision": self.precision,
                "Recall": self.recall,
                "F1 Score": self.fscore
            }
            
            # Prepare data for plotting
            metrics_df = pd.DataFrame({
                "Model": models * 4,
                "Metric": ["Accuracy"] * 3 + ["Precision"] * 3 + ["Recall"] * 3 + ["F1 Score"] * 3,
                "Value": self.accuracy + self.precision + self.recall + self.fscore
            })
            
            # For AutoEncoder, replace None values with 0 for visualization
            metrics_df["Value"] = metrics_df["Value"].fillna(0)
            
            # Create grouped bar chart
            width = 0.2
            x = np.arange(len(models))
            
            # Plot each metric
            colors = [NAVY_BLUE, "#4CAF50", "#FFC107", "#FF5722"]
            for i, (metric, color) in enumerate(zip(["Accuracy", "Precision", "Recall", "F1 Score"], colors)):
                metric_data = metrics_df[metrics_df["Metric"] == metric]["Value"].values
                ax.bar(x + width * (i - 1.5), metric_data, width, label=metric, color=color)
            
            # Customize the chart
            ax.set_title("Model Performance Comparison", fontsize=16, color=NAVY_BLUE)
            ax.set_xlabel("Models", fontsize=14, color=NAVY_BLUE)
            ax.set_ylabel("Score (%)", fontsize=14, color=NAVY_BLUE)
            ax.set_xticks(x)
            ax.set_xticklabels(models)
            ax.legend()
            ax.set_ylim(0, 105)  # Set y-axis limit
            ax.grid(axis='y', linestyle='--', alpha=0.7)
            
            # Style the plot
            ax.set_facecolor(CREAM)
            for spine in ax.spines.values():
                spine.set_color(NAVY_BLUE)
            
            ax.tick_params(colors=NAVY_BLUE)
            
            # Display in the app
            canvas = FigureCanvasTkAgg(fig, self.results_viz_frame)
            canvas.draw()
            canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to generate comparison graph: {str(e)}")
    
    def show_comparison_table(self):
        if len(self.accuracy) < 3:
            messagebox.showwarning("Warning", "Please train all models first!")
            return
        
        try:
            # Clear previous widgets
            for widget in self.results_viz_frame.winfo_children():
                widget.destroy()
            
            # Create frame for table
            table_frame = tk.Frame(self.results_viz_frame, bg=CREAM_LIGHT)
            table_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
            
            # Define table headers and data
            headers = ["Model", "Accuracy (%)", "Precision (%)", "Recall (%)", "F1 Score (%)"]
            models = ["AutoEncoder", "Decision Tree", "DNN"]
            
            # Handle None values for AutoEncoder metrics
            precision_vals = [self.precision[i] if self.precision[i] is not None else "N/A" for i in range(len(self.precision))]
            recall_vals = [self.recall[i] if self.recall[i] is not None else "N/A" for i in range(len(self.recall))]
            fscore_vals = [self.fscore[i] if self.fscore[i] is not None else "N/A" for i in range(len(self.fscore))]
            
            # Create header row
            for j, header in enumerate(headers):
                header_label = tk.Label(table_frame, text=header, font=("Arial", 12, "bold"), bg=NAVY_BLUE, fg=CREAM, padx=10, pady=5)
                header_label.grid(row=0, column=j, sticky="nsew", padx=1, pady=1)
            
            # Create data rows
            for i in range(len(models)):
                # Model name
                model_label = tk.Label(table_frame, text=models[i], font=("Arial", 12), bg=NAVY_BLUE_LIGHT, fg=CREAM, padx=10, pady=5)
                model_label.grid(row=i+1, column=0, sticky="nsew", padx=1, pady=1)
                
                # Accuracy
                accuracy_label = tk.Label(table_frame, text=f"{self.accuracy[i]:.2f}" if isinstance(self.accuracy[i], (int, float)) else "N/A", 
                                      font=("Arial", 12), bg=CREAM, fg=NAVY_BLUE, padx=10, pady=5)
                accuracy_label.grid(row=i+1, column=1, sticky="nsew", padx=1, pady=1)
                
                # Precision
                precision_label = tk.Label(table_frame, text=f"{precision_vals[i]:.2f}" if isinstance(precision_vals[i], (int, float)) else precision_vals[i], 
                                       font=("Arial", 12), bg=CREAM, fg=NAVY_BLUE, padx=10, pady=5)
                precision_label.grid(row=i+1, column=2, sticky="nsew", padx=1, pady=1)
                
                # Recall
                recall_label = tk.Label(table_frame, text=f"{recall_vals[i]:.2f}" if isinstance(recall_vals[i], (int, float)) else recall_vals[i], 
                                    font=("Arial", 12), bg=CREAM, fg=NAVY_BLUE, padx=10, pady=5)
                recall_label.grid(row=i+1, column=3, sticky="nsew", padx=1, pady=1)
                
                # F1 Score
                fscore_label = tk.Label(table_frame, text=f"{fscore_vals[i]:.2f}" if isinstance(fscore_vals[i], (int, float)) else fscore_vals[i], 
                                    font=("Arial", 12), bg=CREAM, fg=NAVY_BLUE, padx=10, pady=5)
                fscore_label.grid(row=i+1, column=4, sticky="nsew", padx=1, pady=1)
            
            # Configure grid weights
            for i in range(len(models) + 1):
                table_frame.grid_rowconfigure(i, weight=1)
            for j in range(len(headers)):
                table_frame.grid_columnconfigure(j, weight=1)
                
            # Add export button
            export_btn = tk.Button(self.results_viz_frame, text="Export Results", 
                              command=self.export_results, 
                              bg=NAVY_BLUE, fg=CREAM, 
                              font=("Arial", 12), 
                              padx=10, pady=5)
            export_btn.pack(pady=20)
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to generate comparison table: {str(e)}")
    
    def export_results(self):
        if len(self.accuracy) < 3:
            messagebox.showwarning("Warning", "No results to export!")
            return
        
        try:
            # Create HTML content
            html_content = f"""
            <!DOCTYPE html>
            <html>
            <head>
                <title>Cyber Threat Detection Results</title>
                <style>
                    body {{ font-family: Arial, sans-serif; margin: 20px; background-color: {CREAM_LIGHT}; color: {NAVY_BLUE}; }}
                    h1, h2 {{ color: {NAVY_BLUE}; }}
                    table {{ border-collapse: collapse; width: 80%; margin: 20px auto; }}
                    th {{ background-color: {NAVY_BLUE}; color: {CREAM}; padding: 12px; text-align: left; }}
                    td {{ background-color: {CREAM}; padding: 8px; border: 1px solid {NAVY_BLUE_LIGHT}; }}
                    tr:nth-child(even) {{ background-color: {CREAM_LIGHT}; }}
                </style>
            </head>
            <body>
                <h1>Cyber Threat Detection Results</h1>
                <h2>Model Performance Comparison</h2>
                <table>
                    <tr>
                        <th>Model</th>
                        <th>Accuracy (%)</th>
                        <th>Precision (%)</th>
                        <th>Recall (%)</th>
                        <th>F1 Score (%)</th>
                    </tr>
            """
            
            # Add model rows
            models = ["AutoEncoder", "Decision Tree", "DNN"]
            for i in range(len(models)):
                precision_val = f"{self.precision[i]:.2f}" if self.precision[i] is not None else "N/A"
                recall_val = f"{self.recall[i]:.2f}" if self.recall[i] is not None else "N/A"
                fscore_val = f"{self.fscore[i]:.2f}" if self.fscore[i] is not None else "N/A"
                
                html_content += f"""
                    <tr>
                        <td>{models[i]}</td>
                        <td>{self.accuracy[i]:.2f}</td>
                        <td>{precision_val}</td>
                        <td>{recall_val}</td>
                        <td>{fscore_val}</td>
                    </tr>
                """
            
            # Close HTML
            html_content += """
                </table>
                <p>Generated by Analysing and Detecting Cyber Threats</p>
            </body>
            </html>
            """
            
            # Save file
            filename = filedialog.asksaveasfilename(defaultextension=".html", filetypes=[("HTML files", "*.html"), ("All files", "*.*")])
            
            if filename:
                with open(filename, "w") as f:
                    f.write(html_content)
                
                messagebox.showinfo("Success", f"Results exported to {filename}")
                
                # Open the file in browser
                import webbrowser
                webbrowser.open(filename)
                
        except Exception as e:
            messagebox.showerror("Error", f"Failed to export results: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    app = CyberThreatDetectionApp(root)
    root.mainloop()